In [10]:
import cv2 as cv
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [11]:
def calc_angle(a,b,c):
    a,b,c = np.array(a), np.array(b), np.array(c)
    ba = a-b
    bc = c-b 
    dot_product = np.dot(ba,bc)
    mag_ba = np.linalg.norm(ba)
    mag_bc = np.linalg.norm(bc)
    cos_theta = dot_product/(mag_ba*mag_bc)
    angle_radian = np.arccos(cos_theta)
    angle_deg = np.degrees(angle_radian)

    if angle_deg>180:
        angle_deg=360-angle_deg
    return angle_deg

In [16]:
cam = cv.VideoCapture(0)


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cam.read()

        # re-color image to RGB for processing by mediapipe
        img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        # changing to read-only to reduce computation complexity
        img.flags.writeable = False
        results = pose.process(img) # processing to make detections
        img.flags.writeable = True

        #revert color back to BGR for OpenCV rendering
        img = cv.cvtColor(img, cv.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            # calculating elbow angle b/w shoulder and wrist
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            angle = calc_angle(shoulder, elbow, wrist)

            # displaying the elbow angle in live-feed
            location = tuple(np.multiply(elbow, [640,480]).astype(int))
            cv.putText(img, str(angle), location, cv.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,0,0), thickness=2)
            
            # print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)        


        cv.imshow("Webcam frame",img)

        if cv.waitKey(1) & 0xFF == ord("q"):
            break

    cam.release()
    cv.destroyAllWindows()



In [15]:
img.shape

(480, 640, 3)

<img src="https://sigmoidal.ai/wp-content/uploads/2023/07/Pose-Landmarker-Model-1024x517.jpg" style="height:300px">